# 문자를 숫자로 표현하는 방법

- [Tokenization](#Tokenization)
    - [Spacing](#Spacing)
    - [Character tokenization](#Character-tokenization)
    - [N-gram](#N-gram)
    - [BPE (Byte Pair Encoding)](#BPE-(Byte-Pair-Encoding))
    - [Pre-Trained Tokenizer](#Pre-Trained-Tokenizer)
- [Embedding](#Embedding)
    - [One-hot Encoding](#One-hot-Encoding)

# <font color='CC3D3D'>Tokenization

In [1]:
S1 = '나는 책상 위에 사과를 먹었다'       
S2 = '알고 보니 그 사과는 Jason 것이었다' 
S3 = '그래서 Jason에게 사과를 했다'

S4 = '나는 책상 위에 배를 먹었다'

# Spacing

In [2]:
print(S1.split())
# ['나는', '책상', '위에', '사과를', '먹었다']

print(S2.split())
# ['알고', '보니', '그', '사과는', 'Jason', '것이었다']

print(S3.split()) 
# ['그래서', 'Jason에게', '사과를', '했다']

['나는', '책상', '위에', '사과를', '먹었다']
['알고', '보니', '그', '사과는', 'Jason', '것이었다']
['그래서', 'Jason에게', '사과를', '했다']


In [3]:
print(list(S1))

['나', '는', ' ', '책', '상', ' ', '위', '에', ' ', '사', '과', '를', ' ', '먹', '었', '다']


In [4]:
token2idx = {}
index = 0

for sentence in [S1, S2, S3]:
    tokens = sentence.split()
    for token in tokens:
        if token2idx.get(token) == None:  # get(x) : x라는 Key에 대응되는 Value를 돌려준다.
            token2idx[token] = index
            index += 1

print(token2idx)


{'나는': 0, '책상': 1, '위에': 2, '사과를': 3, '먹었다': 4, '알고': 5, '보니': 6, '그': 7, '사과는': 8, 'Jason': 9, '것이었다': 10, '그래서': 11, 'Jason에게': 12, '했다': 13}


In [5]:
def indexed_sentence(sentence):
    return [token2idx[token] for token in sentence]
    
S1_i = indexed_sentence(S1.split())
print(S1_i)
S2_i = indexed_sentence(S2.split())
print(S2_i)
S3_i = indexed_sentence(S3.split())
print(S3_i)

[0, 1, 2, 3, 4]
[5, 6, 7, 8, 9, 10]
[11, 12, 3, 13]


### Out-of-Vocabulary(OOV)

In [6]:
S4 = '나는 책상 위에 배를 먹었다'

indexed_sentence(S4.split())
# KeyError: '배를'

KeyError: '배를'

In [7]:
S4 = '나는 책상 위에 배를 먹었다'

# 기존 token 사전에 <unk> token 추가
token2idx = {t : i+1 for t, i in token2idx.items()}  # 인덱스 번호가 하나씩 뒤로 밀림
token2idx['<unk>'] = 0

# token이 없을 경우, <unk> token의 0을 치환
def indexed_sentence_unk(sentence):
    return [ token2idx.get(token, token2idx['<unk>']) for token in sentence ]  # get(찾고자 하는 키 값, 없을 때 넣을 값)

indexed_sentence_unk(S4.split())

[1, 2, 3, 0, 5]

# Character tokenization

In [8]:
# 영어 unicode
print([chr(k) for k in range(65, 91)]) # 영어 대문자
# ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
print([chr(k) for k in range(97, 123)]) # 영어 소문자
# ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# 특수 문자 및 숫자 unicode
print([chr(k) for k in range(32, 48)])
# [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/']
print([chr(k) for k in range(58, 65)])
# [':', ';', '<', '=', '>', '?', '@']
print([chr(k) for k in range(91, 97)])
# ['[', '\\', ']', '^', '_', '`']
print([chr(k) for k in range(123, 127)])
# ['{', '|', '}', '~']
print([chr(k) for k in range(48, 58)])
# ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/']
[':', ';', '<', '=', '>', '?', '@']
['[', '\\', ']', '^', '_', '`']
['{', '|', '}', '~']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [9]:
# 한국어 unicode

print([chr(k) for k in range(int('0xAC00',16), int('0xD7A3',16) + 1)]) # 모든 완성형 한글 11,172자
# ['가', '각', '갂', '갃', '간', ... '힡', '힢', '힣']
print([chr(k) for k in range(int('0x3131',16), int('0x3163',16) + 1)]) # 자모 
# ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', ..., 'ㅢ', 'ㅣ']

['가', '각', '갂', '갃', '간', '갅', '갆', '갇', '갈', '갉', '갊', '갋', '갌', '갍', '갎', '갏', '감', '갑', '값', '갓', '갔', '강', '갖', '갗', '갘', '같', '갚', '갛', '개', '객', '갞', '갟', '갠', '갡', '갢', '갣', '갤', '갥', '갦', '갧', '갨', '갩', '갪', '갫', '갬', '갭', '갮', '갯', '갰', '갱', '갲', '갳', '갴', '갵', '갶', '갷', '갸', '갹', '갺', '갻', '갼', '갽', '갾', '갿', '걀', '걁', '걂', '걃', '걄', '걅', '걆', '걇', '걈', '걉', '걊', '걋', '걌', '걍', '걎', '걏', '걐', '걑', '걒', '걓', '걔', '걕', '걖', '걗', '걘', '걙', '걚', '걛', '걜', '걝', '걞', '걟', '걠', '걡', '걢', '걣', '걤', '걥', '걦', '걧', '걨', '걩', '걪', '걫', '걬', '걭', '걮', '걯', '거', '걱', '걲', '걳', '건', '걵', '걶', '걷', '걸', '걹', '걺', '걻', '걼', '걽', '걾', '걿', '검', '겁', '겂', '것', '겄', '겅', '겆', '겇', '겈', '겉', '겊', '겋', '게', '겍', '겎', '겏', '겐', '겑', '겒', '겓', '겔', '겕', '겖', '겗', '겘', '겙', '겚', '겛', '겜', '겝', '겞', '겟', '겠', '겡', '겢', '겣', '겤', '겥', '겦', '겧', '겨', '격', '겪', '겫', '견', '겭', '겮', '겯', '결', '겱', '겲', '겳', '겴', '겵', '겶', '겷', '겸', '겹', '겺', '겻', '겼', '경', '겾', '겿', '곀', '곁', '곂', '곃', '계', '곅', '곆', '곇',

In [10]:
idx2char = {0:'<pad>', 1:'<unk>'}

srt_idx = len(idx2char)
for x in range(32, 127):  # 영어, 특수문자, 숫자 unicode
    idx2char.update({srt_idx : chr(x)})
    srt_idx += 1

# 한글 추가는 밑의 코드를 실행합니다.
for x in range(int('0x3131',16), int('0x3163',16) + 1):
    idx2char.update({srt_idx : chr(x)})
    srt_idx += 1

for x in range(int('0xAC00',16), int('0xD7A3',16) + 1):
    idx2char.update({srt_idx : chr(x)})
    srt_idx += 1

char2idx = {v:k for k,v in idx2char.items()}

In [11]:
print(idx2char)

{0: '<pad>', 1: '<unk>', 2: ' ', 3: '!', 4: '"', 5: '#', 6: '$', 7: '%', 8: '&', 9: "'", 10: '(', 11: ')', 12: '*', 13: '+', 14: ',', 15: '-', 16: '.', 17: '/', 18: '0', 19: '1', 20: '2', 21: '3', 22: '4', 23: '5', 24: '6', 25: '7', 26: '8', 27: '9', 28: ':', 29: ';', 30: '<', 31: '=', 32: '>', 33: '?', 34: '@', 35: 'A', 36: 'B', 37: 'C', 38: 'D', 39: 'E', 40: 'F', 41: 'G', 42: 'H', 43: 'I', 44: 'J', 45: 'K', 46: 'L', 47: 'M', 48: 'N', 49: 'O', 50: 'P', 51: 'Q', 52: 'R', 53: 'S', 54: 'T', 55: 'U', 56: 'V', 57: 'W', 58: 'X', 59: 'Y', 60: 'Z', 61: '[', 62: '\\', 63: ']', 64: '^', 65: '_', 66: '`', 67: 'a', 68: 'b', 69: 'c', 70: 'd', 71: 'e', 72: 'f', 73: 'g', 74: 'h', 75: 'i', 76: 'j', 77: 'k', 78: 'l', 79: 'm', 80: 'n', 81: 'o', 82: 'p', 83: 'q', 84: 'r', 85: 's', 86: 't', 87: 'u', 88: 'v', 89: 'w', 90: 'x', 91: 'y', 92: 'z', 93: '{', 94: '|', 95: '}', 96: '~', 97: 'ㄱ', 98: 'ㄲ', 99: 'ㄳ', 100: 'ㄴ', 101: 'ㄵ', 102: 'ㄶ', 103: 'ㄷ', 104: 'ㄸ', 105: 'ㄹ', 106: 'ㄺ', 107: 'ㄻ', 108: 'ㄼ', 109: 'ㄽ', 

In [12]:
print([char2idx.get(c,0) for c in '그래서 Jason에게 사과를 했다'])
# [1324, 1832, 2, 8409, 5461, 2, 7064, 6756, 2, 5440, 400, 3600, 2, 3789, 6748, 1912]
print([char2idx.get(c,0) for c in 'ㅇㅋ! ㄱㅅㄱㅅ'])
# [119, 123, 3, 2, 97, 117, 97, 117]

[652, 3116, 5552, 2, 44, 67, 85, 81, 80, 6756, 288, 2, 5440, 400, 3600, 2, 10780, 1912]
[119, 123, 3, 2, 97, 117, 97, 117]


# N-gram

In [13]:
# character 단위 n-gram

S1 = '나는 책상 위에 사과를 먹었다'

print([S1[i:i+1] for i in range(len(S1))]) # uni-gram
#['나', '는', ' ', '책', '상', ' ', '위', '에', ' ', '사', '과', '를', ' ', '먹', '었', '다']

print([S1[i:i+2] for i in range(len(S1))]) # bi -gram
#['나는', '는 ', ' 책', '책상', '상 ', ' 위', '위에', '에 ', ' 사', '사과', '과를', '를 ', ' 먹', '먹었', '었다', '다']

print([S1[i:i+3] for i in range(len(S1))]) # tri-gram
#['나는 ', '는 책', ' 책상', '책상 ', '상 위', ' 위에', '위에 ', '에 사', ' 사과', '사과를', '과를 ', '를 먹', ' 먹었', '먹었다', '었다', '다']

['나', '는', ' ', '책', '상', ' ', '위', '에', ' ', '사', '과', '를', ' ', '먹', '었', '다']
['나는', '는 ', ' 책', '책상', '상 ', ' 위', '위에', '에 ', ' 사', '사과', '과를', '를 ', ' 먹', '먹었', '었다', '다']
['나는 ', '는 책', ' 책상', '책상 ', '상 위', ' 위에', '위에 ', '에 사', ' 사과', '사과를', '과를 ', '를 먹', ' 먹었', '먹었다', '었다', '다']


In [14]:
# spacing 단위 n-gram

S5 = 'I am dying to play the game'
S5_sp = S5.split()

print([" ".join(S5_sp[i:i+1]) for i in range(len(S5_sp))]) # uni-gram
# ['I', 'am', 'dying', 'to', 'play', 'the', 'game']

print([" ".join(S5_sp[i:i+2]) for i in range(len(S5_sp))]) # bi -gram
# ['I am', 'am dying', 'dying to', 'to play', 'play the', 'the game', 'game']

print([" ".join(S5_sp[i:i+3]) for i in range(len(S5_sp))]) # tri-gram
# ['I am dying', 'am dying to', 'dying to play', 'to play the', 'play the game', 'the game', 'game']

['I', 'am', 'dying', 'to', 'play', 'the', 'game']
['I am', 'am dying', 'dying to', 'to play', 'play the', 'the game', 'game']
['I am dying', 'am dying to', 'dying to play', 'to play the', 'play the game', 'the game', 'game']


# BPE (Byte Pair Encoding)

In [15]:
# Algorithm 1 : Learn BPE operations

import re, collections

def get_stats(vocab):  # 연결되어 나오는 단어의 횟수
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {'l o w </w>' : 5, 'l o w e r </w>' : 2, 'n e w e s t </w>' : 6, 'w i d e s t </w>' : 3}
num_merges = 10

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(f'Step {i + 1}')
    print(best)
    print(vocab)
    print('\n')

Step 1
('e', 's')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


Step 2
('es', 't')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


Step 3
('est', '</w>')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}


Step 4
('l', 'o')
{'lo w </w>': 5, 'lo w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}


Step 5
('lo', 'w')
{'low </w>': 5, 'low e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}


Step 6
('n', 'e')
{'low </w>': 5, 'low e r </w>': 2, 'ne w est</w>': 6, 'w i d est</w>': 3}


Step 7
('ne', 'w')
{'low </w>': 5, 'low e r </w>': 2, 'new est</w>': 6, 'w i d est</w>': 3}


Step 8
('new', 'est</w>')
{'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


Step 9
('low', '</w>')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


Step 10
('w', 'i')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}




In [16]:
S1 = '나는 책상 위에 사과를 먹었다'       
S2 = '알고 보니 그 사과는 Jason 것이었다' 
S3 = '그래서 Jason에게 사과를 했다'

token_counts = {}
index = 0

for sentence in [S1, S2, S3]:
    tokens = sentence.split()
    for token in tokens:
        if token_counts.get(token) == None:
            token_counts[token] = 1
        else:
            token_counts[token] += 1

token_counts = {" ".join(token) : counts for token, counts in token_counts.items()}
print(token_counts)

{'나 는': 1, '책 상': 1, '위 에': 1, '사 과 를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사 과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


In [17]:
num_merges = 10

for i in range(num_merges):
    pairs = get_stats(token_counts)
    best = max(pairs, key=pairs.get)
    token_counts = merge_vocab(best, token_counts)
    print(f'Step {i + 1}')
    print(best)
    print(token_counts)
    print('\n')

Step 1
('사', '과')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과 를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


Step 2
('사과', '를')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


Step 3
('었', '다')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


Step 4
('J', 'a')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'Ja s o n': 1, '것 이 었다': 1, '그 래 서': 1, 'Ja s o n 에 게': 1, '했 다': 1}


Step 5
('Ja', 's')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'Jas o n': 1, '것 이 었다': 1, '그 래 서': 1, 'Jas o n 에 게': 1, '했 다': 1}


Step 6
('Jas', 'o')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 

# Pre-Trained Tokenizer

In [1]:
 #! pip install transformers

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # 연구자가 미리 학습해둔 Tokenizer
print(len(tokenizer.vocab))

30522


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentence = "My dog is cute. He likes playing"
print(tokenizer.tokenize(sentence))

['my', 'dog', 'is', 'cute', '.', 'he', 'likes', 'playing']


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')  # 다양한 언어를 담고 있는 다른 데이터에서 학습한 모델
print(len(tokenizer.vocab))
print(tokenizer.tokenize(sentence))
# '##e'에서 ##은 바로 이어지는 Token을 의미


105879
['my', 'dog', 'is', 'cut', '##e', '.', 'he', 'likes', 'playing']


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')  # 한국어도 포함 
sentence = "나는 책상 위에 사과를 먹었다. 알고 보니 그 사과는 Jason 것이었다. 그래서 Jason에게 사과를 했다."
print(tokenizer.tokenize(sentence))
# 제한된 Vocab사이즈에서 최적의 Token을 구해야 하므로 Token이 짧아짐

['나는', 'ᄎ', '##ᅢᆨ', '##상', '위에', 'ᄉ', '##ᅡ', '##과', '##를', 'ᄆ', '##ᅥ', '##ᆨ', '##었다', '.', '알', '##고', 'ᄇ', '##ᅩ', '##니', '그', 'ᄉ', '##ᅡ', '##과', '##는', 'jason', '것이', '##었다', '.', '그', '##래', '##서', 'jason', '##에게', 'ᄉ', '##ᅡ', '##과', '##를', '했다', '.']


# <font color="CC3D3D">Embedding

# One-hot Encoding

In [23]:
S1 = '나는 책상 위에 사과를 먹었다'        
S2 = '알고 보니 그 사과는 Jason 것이었다'  
S3 = '그래서 Jason에게 사과를 했다'       

token2idx = {}
index = 0

for sentence in [S1, S2, S3]:
    tokens = sentence.split()
    for token in tokens:
        if token2idx.get(token) == None:
            token2idx[token] = index
            index += 1

print(token2idx)

# {'나는': 0, '책상': 1, '위에': 2, '사과를': 3, '먹었다': 4, '알고': 5, '보니': 6, '그': 7, '사과는': 8, 'Jason': 9, '것이었다': 10, '그래서': 11, 'Jason에게': 12, '했다': 13}

{'나는': 0, '책상': 1, '위에': 2, '사과를': 3, '먹었다': 4, '알고': 5, '보니': 6, '그': 7, '사과는': 8, 'Jason': 9, '것이었다': 10, '그래서': 11, 'Jason에게': 12, '했다': 13}


In [24]:
V = len(token2idx)

token2vec = [ ([0 if i != idx else 1 for i in range(V)], idx, token) for token, idx in token2idx.items() ] 

for x in token2vec:
    print("\t".join([str(y) for y in x]))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	0	나는
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	1	책상
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	2	위에
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	3	사과를
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]	4	먹었다
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]	5	알고
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]	6	보니
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]	7	그
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]	8	사과는
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]	9	Jason
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]	10	것이었다
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]	11	그래서
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]	12	Jason에게
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]	13	했다


In [25]:
# python numpy를 이용하여 문장을 one-hot encoding으로 바꾸는 방법
import numpy as np
# default_zero = np.zeros((1,V))

for sentence in [S1, S2, S3]:
    onehot_s = []
    tokens = sentence.split()
    for token in tokens:
        if token2idx.get(token) != None:
            vector = np.zeros((1,V))  # (1, v)차원의 0 행렬을 만듦
            vector[:,token2idx[token]] = 1
            #print(vector)
            onehot_s.append(vector)
        else:
            print("UNK")

    print(f"{sentence} : ")        
    print(np.concatenate(onehot_s, axis = 0))
    print('\n')

나는 책상 위에 사과를 먹었다 : 
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


알고 보니 그 사과는 Jason 것이었다 : 
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


그래서 Jason에게 사과를 했다 : 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


